# Fund Strategy - Signal Event 0002

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2015-12-31'
ed = '2026-02-13'

In [3]:
instrument_list = ['513030.SH']

In [4]:
sql = f"""
SELECT
    date,
    instrument,
    m_ta_macd_dea(close) AS DEA,
    m_ta_macd_dif(close) AS DIF,
    IF(DIF > DEA AND m_lag(DIF, 1) < m_lag(DEA, 1), 1, 0) AS signal_buy,
    IF(DIF < DEA AND m_lag(DIF, 1) > m_lag(DEA, 1), 1, 0) AS signal_sell,
FROM cn_fund_bar1d
WHERE instrument IN {str(tuple(instrument_list))}
QUALIFY (signal_buy = 1 OR signal_sell = 1)
"""

In [5]:
df = dai.query(sql, filters = {"date":[sd, ed]}).df()
df

,date,instrument,DEA,DIF,signal_buy,signal_sell
0,2024-07-01,513030.SH,-0.005914,-0.005662,1,0
1,2024-07-22,513030.SH,0.005922,0.005218,0,1
2,2024-08-15,513030.SH,-0.015769,-0.014781,1,0
3,2024-09-06,513030.SH,0.010138,0.009550,0,1
4,2024-09-19,513030.SH,0.005847,0.006218,1,0
...,...,...,...,...,...,...
165,2024-03-20,513030.SH,0.018071,0.017688,0,1
166,2024-03-21,513030.SH,0.018227,0.018849,1,0
167,2024-04-03,513030.SH,0.021152,0.020905,0,1
168,2024-05-06,513030.SH,-0.002401,-0.002373,1,0


## 3. Backtest Strategy

In [6]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.holding_days = 3
    context.is_trade_by_weight = True
    context.weight = 1 / len(context.data['instrument'].unique())

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return

    instruments_buy  = set(df_now[df_now["signal_buy"]  == 1]["instrument"])
    instruments_sell = set(df_now[df_now["signal_sell"] == 1]["instrument"])
    instruments_hold = set(context.get_account_positions().keys())

    for instrument in instruments_buy - instruments_hold:
        if context.is_trade_by_weight:
            context.order_target_percent(instrument, context.weight)
        else:
            context.order(instrument, 100)
    
    for instrument in instruments_sell:
        context.order_target_percent(instrument, 0)

    for instrument in instruments_hold:
        if (data.current_dt - context.get_position(instrument).last_sale_date).days >= context.holding_days:
            context.order_target_percent(instrument, 0)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000 + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-19 15:08:28] [info     ] bigtrader.v34 开始运行 ..
[2026-02-19 15:08:29] [info     ] 2016-04-06, 2025-01-08, , , instruments=1
[2026-02-19 15:08:29] [info     ] bigtrader module V2.2.0
[2026-02-19 15:08:29] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-19 15:08:31] [info     ] backtest done, raw_perf_ds:dai.DataSource("_93022438db274c458aea307a4964d9df")


[2026-02-19 15:08:32] [info     ] bigtrader.v34 运行完成 [3.136s].
